In [85]:
import sys
sys.path.append('..')
import crystalmaths
from crystalmaths.find_matching_angles import find_matching_angles
import itertools
import requests
import numpy as np
import pandas as pd
%matplotlib qt
# from crystalmaths import 


In [86]:
image_filepath = r'C:\Users\lacho\OneDrive - UW\crystalmaths\Local_data\SL_Bornite_011.png'

image_object = crystalmaths.imagetools.ImageHandler(image_filepath=image_filepath)

In [90]:
image_object.image_array.shape

image_object.get_scale_ratio()
image_object.scale_ratio

Enter scalebar length in nanometers.2
length_nm =  2.0
length_pixel = 22.898507903678535


11.449253951839268

In [87]:
image_object.get_planes()

Pick planes using left mouse button. Right click once done. Middle mouse button removes most recent point.
Exited zoom loop
Entered point pick loop
Saved points =  [(153.5, 154.36052727004125), (163.4920761761506, 74.8402543681758)]
Press keyboard button to save points, mouseclick to restart
Entered point pick loop
Saved points =  [(250.50640621012906, 28.626902053479114), (90.63318739171892, 36.120959185592085)]
Press keyboard button to save points, mouseclick to restart


In [88]:
image_object.point_coordinates 

[[95.0, -126.0], [-65.0, -118.0]]

In [91]:
def get_zone_axis(image_object, d_spacing_tolerance, angle_tolerance):
    """
    get_zone_axis will take one image_object which has had two planes
    selected on it. The two points (e.g. d1, d2) will be passed as 
    arguments to get_d, which searches the ACMS online database to
    generate a list of candidate minerals, based on the d-spacing 
    of the selected points from fft image.
    
    Tolerances for search criteria are also passed as float arguments into
    this function to provide cutoffs for difference in d-spacing and in 
    angles. 
    
    get_zone_axis returns a DataFrame with the HKL and mineral name 
    for all sets of matching planes
    """
    angle_compare_list=[]
    p1 = image_object.point_coordinates[0]
    p2 = image_object.point_coordinates[1]
    fft_angle = crystalmaths.angles_from_fft.angle(p1, p2)
    d1 = crystalmaths.distance_from_fft.distance_from_fft(p1, image_object.scale_ratio, image_object.image_array.shape[0])
    d2 = crystalmaths.distance_from_fft.distance_from_fft(p2, image_object.scale_ratio, image_object.image_array.shape[0])
    link = crystalmaths.get_d.make_web_address(d1, d2, tolerance=d_spacing_tolerance)
    link_list = crystalmaths.get_d.compile_links(link)
    query_results = crystalmaths.get_d.get_d(link_list, [d1, d2])
    print(d1, d2)
    for i,result in enumerate(query_results):
        data_df = result[0]
        metadata_df = result[1]
        temp_object = crystalmaths.angles_from_structure_data.AllAnglePairs(data_df, metadata_df)
        result_df = temp_object.result_df
        result_df = find_matching_angles(fft_angle, result_df, angle_tolerance)
        final_df = result_df[result_df['angle match']==True]
#         df[df['Btu']>1500000]
        if final_df.empty:
            pass
        else:
            mineral_name = metadata_df['Mineral_Name'].values
            n = final_df.shape[0]
            mineral_name_list = [mineral_name for i in range(n)]
            final_df.insert(0, 'Mineral_Name', mineral_name_list)
            angle_compare_list.append((final_df, metadata_df))
    return angle_compare_list

awesome_list = get_zone_axis(image_object, 0.01, 5)

1.7158373265012095 2.0098245883038475


ValueError: math domain error

In [84]:
for thing in awesome_list:
    print(thing[0]['Mineral_Name'], thing[0]['zone axis'])



3    [Fluoro-richterite]
Name: Mineral_Name, dtype: object 3    [-1.0, -3.0, 11.0]
Name: zone axis, dtype: object
3    [Fluoro-richterite]
Name: Mineral_Name, dtype: object 3    [-1.0, -3.0, 11.0]
Name: zone axis, dtype: object
3     [Sapphirine]
5     [Sapphirine]
6     [Sapphirine]
18    [Sapphirine]
Name: Mineral_Name, dtype: object 3      [4.0, -4.0, 10.0]
5      [10.0, -1.0, 7.0]
6      [-2.0, -6.0, 7.0]
18    [-4.0, 4.0, -10.0]
Name: zone axis, dtype: object
2     [Ransomite]
13    [Ransomite]
Name: Mineral_Name, dtype: object 2     [-6.0, 3.0, -3.0]
13     [8.0, 2.0, -5.0]
Name: zone axis, dtype: object
7    [Hancockite]
Name: Mineral_Name, dtype: object 7    [1.0, 6.0, -2.0]
Name: zone axis, dtype: object
7    [Epidote-(Pb)]
Name: Mineral_Name, dtype: object 7    [1.0, 6.0, -2.0]
Name: zone axis, dtype: object
4     [Goldichite]
5     [Goldichite]
8     [Goldichite]
16    [Goldichite]
24    [Goldichite]
28    [Goldichite]
32    [Goldichite]
44    [Goldichite]
47    [Goldichite]

Name: Mineral_Name, dtype: object 1    [-14.0, 14.0, -4.0]
6      [22.0, 2.0, -4.0]
Name: zone axis, dtype: object
7     [Wakabayashilite]
36    [Wakabayashilite]
40    [Wakabayashilite]
Name: Mineral_Name, dtype: object 7     [0.0, 0.0, -33.0]
36    [-4.0, 3.0, 29.0]
40     [0.0, 0.0, 33.0]
Name: zone axis, dtype: object
17    [Albite]
Name: Mineral_Name, dtype: object 17    [-8.0, 0.0, -4.0]
Name: zone axis, dtype: object
3     [Dickinsonite-(KMnNa)]
5     [Dickinsonite-(KMnNa)]
8     [Dickinsonite-(KMnNa)]
15    [Dickinsonite-(KMnNa)]
25    [Dickinsonite-(KMnNa)]
33    [Dickinsonite-(KMnNa)]
67    [Dickinsonite-(KMnNa)]
70    [Dickinsonite-(KMnNa)]
71    [Dickinsonite-(KMnNa)]
Name: Mineral_Name, dtype: object 3      [12.0, -20.0, -4.0]
5      [-2.0, -10.0, 14.0]
8       [8.0, 16.0, -12.0]
15       [6.0, 34.0, -2.0]
25      [-21.0, 7.0, -2.0]
33       [18.0, 2.0, -6.0]
67       [-18.0, 6.0, 8.0]
70       [-17.0, 3.0, 8.0]
71    [-16.0, -12.0, 10.0]
Name: zone axis, dtype: object
0  

In [ ]:
for i,result in enumerate(query_results):
    data_df = result[0]
    metadata_df = result[1]
    temp_object = crystalmaths.angles_from_structure_data.AllAnglePairs(data_df, metadata_df)
    new_tuple_list.append(tuple(temp_object.result_df, metadata_df))
# temp_object.result_df.head()

In [ ]:
fake_angle = [temp_object.result_df['angle'].mean()]
result_df = temp_object.result_df
result_df = find_matching_angles(fake_angle, result_df, 2)
result_df

Future Work

# Edwin's code starts from here.

In [ ]:
#This is the function passing all data frame of query results
#Start of all testing and comparison
full_angle_list = []
full_plane1_list = []
full_plane2_list = []
for k in range(len(query_results)):
    plane1_list = []
    plane2_list = []
    df1 = query_results[k][0]
    df11 = df1.loc[df1['D-REF'] == 3.3]
    df12 = df1.loc[df1['D-REF'] == 3.5]
    df_p = query_results[k][1]
    a = []
    lattice_parameter = [float(df_p['a'][0]), float(df_p['b'][0]), float(df_p['c'][0])]
    alpha = float(df_p['alpha'][0])
    beta = float(df_p['beta'][0])
    gamma = float(df_p['gamma'][0])
    for i in range(df11.shape[0]):
        for j in range(df12.shape[0]):
            plane1 = [df11['H'].iloc[i], df11['K'].iloc[i], df11['L'].iloc[i]]
            plane2 = [df12['H'].iloc[j], df12['K'].iloc[j], df12['L'].iloc[j]]
            a.append(crystalmaths.temp_angles_from_structure_data_Edwin.angle_for_all(plane1, plane2, lattice_parameter, alpha, beta, gamma))
            plane1_list.append(plane1)
            plane2_list.append(plane2)
#     print(a)
    full_angle_list.append(a)
    full_plane1_list.append(plane1_list)
    full_plane2_list.append(plane2_list)
    
full_angle_list

In [ ]:
#Given angles and find zone axis
angle_list1 = [11, 19, 34, 42]
tolerance = 1
matching_angle_list = []
for i in range(5):
    angle_list2 = full_angle_list[i]
    c = crystalmaths.find_matching_angles.find_matching_angles(angle_list1, angle_list2, tolerance)
    matching_angle_list.append(c)
    
interested_plane1 = []
interested_plane2 = []
zone_axis_dic = {}
for j in range(len(matching_angle_list)):
    zone_axis = []
    for k in range(len(matching_angle_list[j])):
        for l in range(len(full_angle_list[j])):
            if matching_angle_list[j][k][1] == full_angle_list[j][l]:
                interested_plane1.append(full_plane1_list[j][l])
                interested_plane2.append(full_plane2_list[j][l])
                zone_axis.append(np.cross(full_plane1_list[j][l], full_plane2_list[j][l]))  
                zone_axis_dic[query_results[j][1]['Mineral_Name'][0]] = zone_axis
print(zone_axis_dic)

            
            
